# 01 - Introduction

Dictionaries are everywhere in Python. All of the following are dictionaries: modules, classes, objects (class instances), scopes, sets, your own dictionaries.

Here are two main points:
- dictionary keys must be **hashable**.
- dictionary key order is **maintained** (in order of insertion).

# 02 - Associative Arrays

Let's say we have the following list:
```python
persons = [John, Eric, Michael, Graham]
```
where each name is a `Person` object. How do we retrieve `Michael`? We have to remember its index in the list which is 2. Therefore the index behaves like a key to the different objects in the list. Of course, remembering numbers is not a great idea. So, let's consider this following list: 
```python
persons = [
    ('john', John),
    ('eric', Eric),
    ('michael', Michael),
    ('graham', Graham)
]
```
To get the `Michael` Person object, we need look up the key `'michael'` and return the associated value. This is better, but we still have to scan through the entire list to find our key. 

How could we improve it? Let's consider another approach. We'll split our list of tuples into two lists:
```python
keys = ['john', 'eric', 'michael', 'graham']
persons = [John, Eric, Michael, Graham]
```
Then, we can define a function `h` such that `h('john') -> 0`, `h('eric') -> 1`, `h('michael') -> 2` and `h('graham') -> 3`.

To get Michael now, we would perform `persons[h('michael')] --> persons[2] --> Michael`. 

So what are **associative arrays**?

They are an **abstract data structure** that associates (unique) **keys** to **values**. 

They are also called **maps** or **dictionaries**.

They can be implemented in a number of concrete ways.

Any of these ways should support: 
- **adding/removing key-value pairs**
- **modifying an associated value**
- **looking up a value via its key** (the more efficient this step is, the more efficient our dictionaries will be).

# 03 - Hash Maps (Hash Tables)

This is one concrete implementation of associative arrays.

Suppose we have an array of 7 slots (labelled 0-6), initially containing nothing. This is our hash map/hash table/dictionary. We want to store the aforementioned Person objects.

We'll define a function that'll return an integer value given a string e.g 'john'.

We need to ensure:
- the value will be unique for each given string (this is the hard bit)
- is between 0 and 6
- always returns the same integer for the same string (deterministic)

A **hash function** is a function in the mathematical sense such that if `x = y`, `f(x) = f(y)`.

It maps a set (domain, D) of arbitrary size (possibly infinite) to another (smaller) set of fixed size (range, R)

`h: D -> R where X(R) < X(D)` (`X` is chi and it's the cardinality of the set, i.e. the number of elements in the set). 

Let's drop our first requirement that we mentioned above by now allowing getting the **same output** for **different keys**, e.g. `h('john') -> 15` and `h('michael') -> 15`.

Let's write a simple hash function that's based on the **length** of the key:

In [2]:
def h(key, num_slots):
    return len(key) % num_slots

print(h('alexander', 11))
print(h('john', 11))
print(h('eric', 11))

9
4
4


Here we have a **collision**.

Let's write another hash function that uses the ord function (ASCII value from character):

In [7]:
def h(key, num_slots):
    total = sum(ord(c) for c in key)
    return total % num_slots

print(h('alexander', 5))
print(h('john', 5))
print(h('eric', 5))
print(h('michael', 5))

3
1
4
3


**Dealing with Collisions**

There are numerous ways but we'll show two. The first is:

**Chaining**:

We'll store the collided key-value pair as a list of lists/tuples. Using the last example:
```python
0 -> 
1 -> ['john', John]
2 -> 
3 -> [['alexander', Alexander], ['michael', Michael]]
4 -> ['eric', Eric]
```
Now, when `h['michael']` returns 3, we need to iterate through each item until we find the list whose first element is `'michael'`. 

**Probing (linear)**

Now, each key returns a value and a probe sequence. This sequence value starts from the value and increases incrementally until we've covered all the numbers. This probe sequence is also deterministic, so if `h('alexander', 5) -> 3`, then the probe sequence associated with `3` is **always** `3 -> 4 -> 0 -> 1 -> 2`.

Let's apply it to our case above:
```
h('alexander', 5) -> 3 : Sequence 3 -> 4 -> 0 -> 1 -> 2
h('john', 5) ->      1 : Sequence 1 -> 2 -> 3 -> 4 -> 0
h('eric', 5) ->      4 : Sequence 4 -> 0 -> 1 -> 2 -> 3
h('michael', 5) ->   3 : Sequence 3 -> 4 -> 0 -> 1 -> 2
h('graham', 5) ->    4 : Sequence 4 -> 0 -> 1 -> 2 -> 3
```

The first three are fine; we have no collisions.
```python
0 -> 
1 -> ['john', John]
2 -> 
3 -> ['alexander', Alexander]
4 -> ['eric', Eric]
```
But for `'michael'`, we have a collision. What we do is we go through the probe sequence until we land on a value that is not taken. In the sequence `3 -> 4 -> 0 -> 1 -> 2`, `3` and `4` are taken so `'michael'` gets a value of `0`.

For `'graham'`, we also have a collision. In its sequence of `4 -> 0 -> 1 -> 2 -> 3`, `4`, `0` and `1` are all taken so it gets a value of `2`
```python
0 -> ['michael', Michael]
1 -> ['john', John]
2 -> ['graham', Graham]
3 -> ['alexander', Alexander]
4 -> ['eric', Eric]
```


If we were to look up the key `'michael'` after having added `'alexander'`, we would see if `'michael'` is at position `3`. It isn't, so we continue through the probe sequence. Is it at `4`? No, so move onto `0`, and that's where we find it.

**Sizing Issues**

When it comes to creating a hash table, we start small and grow it as needed. But resizing is expensive because the hash values will change.

Deleting pairs also makes things much more complicated but we won't worry about that for now.

# 04 - Python Dictionaries

To make dictionaries as efficient as possible, two implementations in particular were made: **key sharing** and **compact dictionaries**

#### Key Sharing (PEP 412)

Let's say we have 3 Person objects:

In [8]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

john = Person('John', 78)
eric = Person('Eric', 75)
michael = Person('Michael', 78)

We have three groupings that look like:
```python
john
['name', 'John']
['age', 78]

eric
['name', 'Eric']
['age', 75]

michael
['name', 'Michael']
['age', 78]
```
But since we expect to have multiple instances of the same class, we can expect that the keys e.g. 'name' and 'age' will appear in all three dictionaries. What we can do instead is pool these keys as separate objects which store all the associated values:

```python
            john    eric    michael
'name' -> ['John', 'Eric', 'Michael']
'age' ->  [  78,     75,      75    ]
```

This is also known as a **split-table dictionary**. As you can see the multiple class instances are key sharing which optimises storage.

#### Compact Dictionaries

I am not going to write this subsection out. You can watch from 2:55 in Video 7. Basically, this allows the key order to be the same as insertion order which is achieved by splitting our associative array into two smaller arrays, values and indices.

# 05 - Python's `hash()` Function

This function always returns an `int`.

Also, if `a == b`, then `hash(a) == hash(b)`, **even if `a is not b`**.

But, if `a != b`, then the following is **possible** : `hash(a) == hash(b)`. This is a **hash collision**. 

Regarding the first equality:

In [23]:
t1 = (1, 2, 3)
t2 = (1, 2, 3)

print(t1 == t2)
print(t1 is t2)

True
False


But the hash will be the same because they **compare equal**

In [24]:
hash(t1) == hash(t2)

True

Let's look at the hashed value for different inputs:

In [2]:
list(map(hash, [1, 2, 3, 4]))

[1, 2, 3, 4]

In [4]:
list(map(hash, [1.1, 2.2, 3.3, 4.4]))

[230584300921369601,
 461168601842739202,
 691752902764107779,
 922337203685478404]

In [6]:
list(map(hash, ['hello', 'Python', '!']))

[-3570319062466401029, 2050679169924955308, -1117742727718564197]

We can even hash tuples:

In [7]:
hash((1, 'a', 10.5))

7390257652650833158

But not lists:

In [8]:
hash([1, 2])

TypeError: unhashable type: 'list'

Or sets:

In [9]:
hash({1, 2})

TypeError: unhashable type: 'set'

But we can hash frozen sets:

In [13]:
hash(frozenset({1, 2}))

-1826646154956904602

In other words, **we can only hash immutable types**.

This means we cannot hash a tuple that contains a list either:

In [14]:
hash(('a', 1, [2, 3]))

TypeError: unhashable type: 'list'

But we can hash functions even though we can change metadata, docstrings etc., because we are not mutating the function itself:

In [21]:
def func():
    return None

hash(func)

182101179765

Remember that the hash function is used to determine the position index and hence, the start of the probe sequence.

Let's consider the following `a` which is a **key** in our dictionary `d`:

In [16]:
a = (1, 2, 3)
d = {a: 'this key is a tuple - immutable'}

We want to hash this key `a` which will produce a hash value. This hash value will be mapped to a specifix index in an array which retrieves the desired value.

The above dictionary is allowed because looking up `d[a]` will always look for `a` at the same index.

But let's try `a` as a list:

In [18]:
a = [1, 2, 3]
d = {a: 'this key is a list - mutable'}

TypeError: unhashable type: 'list'

This doesn't work. Why? Let's say we could generate a hash value for it. Then, we mutate `a`, e.g. `a.append(4)`. In Python, the object `a` has remained the same, **but the hash has changed**.

So looking up `d[a]` which calculates `hash(a)` as an intermediary step, will return an index that corresponds to the wrong desired value.

Going back to the earlier example:

In [26]:
t1 = (1, 2, 3)
t2 = (1, 2, 3)

t1 is t2

False

In [28]:
d = {t1: 'this is a tuple'}

print(d[t1])
print(d[t2])

this is a tuple
this is a tuple


Both return the desired value because their hashes are the same even if the objects arent. The reason why it's done this way can be understood by the following:

In [29]:
d = {(1, 2, 3): 'another tuple'}
d[(1, 2, 3)]

'another tuple'

The two `(1, 2, 3)`s will be different objects, but we still need the same desired value.

**Caveat:**

The hash value obtained from Python's `hash()` for a given input will always be the same **for a single run of the program**. It will differ from run to run (which prevents particular types of hacking)

#### 06 - Mini Summaries

##### How Python performs dictionary insertions

This is a simplified explanation without worrying about dictionary resizing or how Python handles deletions.

We start with a key and a value:

1. The key is hashed -> `hash(key) = hash value`.
2. The `hash value` is modded against the **allocated** dictionary size to get the start index in a hash table -> `<hash value> mod <allocated_dict_size> = start index` 
3. A probe sequence is generated which is a sequence of valid indices starting from the start index all the way until the last possible index value.
4. Iterate over the probe sequence. If the slot at the index is empty, store the following pair `(hash value, key, value).` Otherwise we have a collision so continue iterating until we hit an empty slot.

##### How Python performs dictionary lookups

The first three steps are the same.

We start with a key:

1. The key is hashed -> `hash(key) = hash value`.
2. The `hash value` is modded against the **allocated** dictionary size to get the start index in a hash table -> `<hash value> mod <allocated_dict_size> = start index`.
3. A probe sequence is generated which is a sequence of valid indices starting from the start index all the way until the last possible index value.
4. Iterate over the probe sequence.
   - If the slot at the index is empty, the key **doesn't exist**. If slot isn't empty, then for `(hash value, key, value)`, we check if the `hash value` and `key` pair is equal to what we just computed. We check the `hash value` first because it's an integer comparison (very fast).
   - If the hash value and key pair is equal, we found the key, so return the value. If they aren't equal, continue iterating until we find a matching key or an empty slot.